In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlite3 as sql

#### Getting all of the European Countries That are Common in both Datasets

In [ ]:
def getAllEuropeanCountries(LifeData, InternetData, CensusData):
    CountryArr = [];

    tempDF = LifeData.merge(InternetData, how='inner', on='geo');
    tempDF = tempDF.merge(CensusData, how='inner', on='geo');
    tempDF = tempDF.drop_duplicates(subset=['geo'], keep='first');
    CountryArr = tempDF['geo'].to_numpy();

    for i in range(len(CountryArr) - 1):
        if(CountryArr[i] == 'European Union - 27 countries (from 2020)'):
            CountryArr = np.delete(CountryArr, i);

    return CountryArr;

#### Getting Satisfaction Rating for Each Country Listed in the Above Function

In [ ]:
# Gets the Average Satisfaction in Each Country Betwen 2024-2014.
def getSatisfactionData(Data, CountryList):
    Countries = [];

    for i in range(len(CountryList)):
        Countries.append([0] * (2024 - 2014));

    for i in range(len(Data)):

        for j in range(len(CountryList)):
            if(Data['geo'][i] == CountryList[j] and Data['TIME_PERIOD'][i] >= 2014 and Data['isced11'][i] == 'Upper secondary and post-secondary non-tertiary education (levels 3 and 4)' and Data['life_sat'][i] == 'Life as a whole' and Data['age'][i] == '16 years or over'):
                Index = Data['TIME_PERIOD'][i] - 2019;
                Countries[j][Index] = Data['OBS_VALUE'][i];

    return Countries;

#### Getting Internet Access in all Common Countries

In [ ]:
# Gets the Internet Access in Each Country Between 2024-2014
def getInternetAccessData(Data, CountryList):
    Countries = [];

    for i in range(len(CountryList)):
        Countries.append([0] * (2024 - 2014));

    for i in range(len(Data)):
        for j in range(len(CountryList)):
            if(Data['geo'][i] == CountryList[j] and Data['TIME_PERIOD'][i] >= 2014):
                Index = Data['TIME_PERIOD'][i] - 2019;
                Countries[j][Index] = Data['OBS_VALUE'][i];

    return Countries;

In [ ]:
def getCensusData(Data, CountryList):
    Countries = [];

    for i in range(len(CountryList)):
        Countries.append([0] * (2024 - 2014));

    for i in range(len(Data)):
        if Data['sex'][i] == 'Total':
            for j in range(len(CountryList)):
                if(Data['geo'][i] == CountryList[j] and Data['TIME_PERIOD'][i] >= 2014):
                    Index = Data['TIME_PERIOD'][i] - 2019;
                    Countries[j][Index] = Data['OBS_VALUE'][i];
    return Countries;


#### We first Read our Datasets in as CSV and Parse them Pandas Dataframe
#### We then create a Table listing all Countries
#### And then create a Table for each Country, inserting the data into that table

In [ ]:
def main():
    #Our Datasets.
    LifeSatisfactionData = pd.read_csv('../Raw Data/Overall life satisfaction by sex, age and educational attainment.csv');
    InternetData = pd.read_csv('../Raw Data/Households - level of internet access.csv');
    CensusData = pd.read_csv('../Raw Data/Census_2021.csv')

    CountryList = getAllEuropeanCountries(LifeSatisfactionData, InternetData, CensusData);

    #Set up SQL Connection.
    Connection = sql.connect('../Raw Data/HOORAH.db');

    #We make a Table for all Included Countries.
    DF = pd.DataFrame({'Country Name' : CountryList, });
    DF.to_sql('Countries', Connection, if_exists='replace');

    #Parsing our Original Dataset to get what we want.
    ParsedSatisfactionData = getSatisfactionData(LifeSatisfactionData, CountryList);
    ParsedInternetAccessData = getInternetAccessData(InternetData, CountryList)
    ParsedCensusData = getCensusData(CensusData, CountryList)
    YearRange = np.arange(2014, 2024, 1);

    #Formatting a Pandas Dataframe Object.
    Format = ['Year', 'Satisfaction', 'Internet Access', 'Population'];

    for i in range(len(CountryList)):
        DataframeData  = [];

        for j in range(len(ParsedSatisfactionData[i])):
            if(ParsedSatisfactionData[i][j] and ParsedInternetAccessData[i][j]):
                DataframeData.append(np.array([int(YearRange[j]), ParsedSatisfactionData[i][j], ParsedInternetAccessData[i][j], ParsedCensusData[i][j]]));

        #Send Country Data to it's own table hehehehe
        PDF = pd.DataFrame(data=DataframeData, columns=Format);
        PDF.to_sql(CountryList[i], Connection, if_exists='replace');

    #Close Connection.
    Connection.close();


#### 8)

In [ ]:
#Call da main function
main();